In [7]:
!pip install requests
!pip install vincenty

import requests
import json
import pandas as pd
from vincenty import vincenty

query = """query loosNearNeontribe {
    loos(filters:{}, pagination:{limit:13683, page:1}){
    loos{
      id
      name
      location{lat}
      location{lng}
      active
      accessible
      radar
    }
  }
  }"""

url = 'https://www.toiletmap.org.uk/api'
r = requests.post(url, json={'query': query})
print(r.status_code)
print(r.text)

json_data = json.loads(r.text)

df_data = json_data["data"]["loos"]["loos"]

#with open('loo_data.txt', 'w') as outfile:
#    json.dump(df_data, outfile)


  ERROR: Command errored out with exit status 1:
   command: /home/luca.bedogni/dsg/p3venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-vslyz24b/vincenty/setup.py'"'"'; __file__='"'"'/tmp/pip-install-vslyz24b/vincenty/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-hekqndin
       cwd: /tmp/pip-install-vslyz24b/vincenty/
  Complete output (6 lines):
  usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: setup.py --help [cmd1 cmd2 ...]
     or: setup.py --help-commands
     or: setup.py cmd --help
  
  error: invalid command 'bdist_wheel'
  ----------------------------------------
  ERROR: Failed building wheel for vincenty
  Running setup.py clean for vincenty
Failed to build vincenty
    Running setup.py install for vincenty ... done
200
{"data":{"loos":{"loos":

In [2]:
df_loos = pd.json_normalize(df_data,max_level = 2)

In [3]:
df_loos.rename({"location.lat":"lat","location.lng":"lon"},inplace = True, axis = 1)

In [10]:
#for j in range(len(df)):
 #   if row["active"].iloc[j] == false:
  #      print(yes)
   #     break

TypeError: string indices must be integers

In [4]:
df_pois = pd.read_pickle("/shared/accessible_pois.pkl")

In [10]:
df_pois

,id,access,barrier,bicycle,motor_vehicle,opening_hours,wheelchair,amenity,lon,lat,acc_loos_100,acc_loos_250,acc_loos_500,acc_loos_1000,all_loos_100,all_loos_250,all_loos_500,all_loos_1000
99878,99878,permissive,gate,no,no,dawn-dusk,yes,None,-0.152985,51.524358,0,0,0,0,0,0,0,0
104734,104734,NONE,None,None,None,None,yes,None,-1.785876,51.565653,0,0,0,0,0,0,0,0
106213,106213,NONE,None,None,None,None,yes,None,-0.142942,51.525660,0,0,0,0,0,0,0,0
108042,108042,NONE,None,None,None,Mo-We 16:00-23:30; Th-Fr 16:00-01:00; Sa 16:00...,limited,pub,-0.135513,51.523561,0,0,0,0,0,0,0,0
251191,251191,NONE,None,None,None,None,limited,None,-0.715918,51.115444,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8495396114,8495396114,yes,None,None,None,None,yes,None,-2.997905,51.589057,0,0,0,0,0,0,0,0
8495396115,8495396115,yes,None,None,None,None,yes,None,-2.997255,51.589114,0,0,0,0,0,0,0,0
8497470120,8497470120,private,gate,private,private,None,limited,None,-0.135463,51.602958,0,0,0,0,0,0,0,0
8497671325,8497671325,NONE,gate,None,no,24/7,no,None,-4.336600,55.905952,0,0,0,0,0,0,0,0


In [16]:
#df_pois = df_pois.reset_index()

df_pois.loc[:,"acc_loos_100"] = 0
df_pois.loc[:,"acc_loos_250"] = 0
df_pois.loc[:,"acc_loos_500"] = 0
df_pois.loc[:,"acc_loos_1000"] = 0

df_pois.loc[:,"all_loos_100"] = 0
df_pois.loc[:,"all_loos_250"] = 0
df_pois.loc[:,"all_loos_500"] = 0
df_pois.loc[:,"all_loos_1000"] = 0
c = 0
for row,col in df_pois.iterrows():
    c +=1
    if c % 100 == 0:
        print(c)
            
    _ = df_loos[(df_loos["lat"] < col["lat"] + 0.05) & (df_loos["lat"] > col["lat"] - 0.05)&(df_loos["lon"] < col["lon"] + 0.1) & (df_loos["lon"] > col["lon"] - 0.1)]
    
    for row_loos , col_loos in _.iterrows():
    
        
        geo = (col["lat"], col["lon"])
        geo_loos = (col_loos["lat"], col_loos["lon"])
        
        dist = 1000 * vincenty(geo,geo_loos)
        
        if dist <= 100:
            df_pois.loc[row,"all_loos_100"] += 1
            
            if col_loos["accessible"] == True:
                df_pois.loc[row,"acc_loos_100"] += 1
                
        if dist <= 250:
            df_pois.loc[row,"all_loos_250"] += 1
            
            if col_loos["accessible"] == True:
                df_pois.loc[row,"acc_loos_250"] += 1
                
        if dist <= 500:
            df_pois.loc[row,"all_loos_500"] += 1
            
            if col_loos["accessible"] == True:
                df_pois.loc[row,"acc_loos_500"] += 1
                
        if dist <= 1000:
            df_pois.loc[row,"all_loos_1000"] += 1
            
            if col_loos["accessible"] == True:
                df_pois.loc[row,"acc_loos_1000"] += 1
                
        
        
    

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [20]:
df_pois.loc[:,"ratio_100"] = df_pois["acc_loos_100"]/df_pois["all_loos_100"]
df_pois.loc[:,"ratio_250"] = df_pois["acc_loos_250"]/df_pois["all_loos_250"]
df_pois.loc[:,"ratio_500"] = df_pois["acc_loos_500"]/df_pois["all_loos_500"]
df_pois.loc[:,"ratio_1000"] = df_pois["acc_loos_1000"]/df_pois["all_loos_1000"]

df_pois.to_pickle('/shared/accessible_pois_with_toilet_distances.pkl')

In [18]:
df_pois.sort_values(["acc_loos_1000"])

,level_0,index,id,access,barrier,bicycle,motor_vehicle,opening_hours,wheelchair,amenity,...,acc_loos_500,acc_loos_1000,all_loos_100,all_loos_250,all_loos_500,all_loos_1000,ratio_100,ratio_250,ratio_500,ratio_1000
14640,14640,2295895325,2295895325,NONE,None,None,None,None,yes,toilets,...,0,0,1,3,3,7,0.0,0.000000,0.000000,0.000000
14304,14304,2152034615,2152034615,NONE,None,None,None,None,limited,fast_food,...,0,0,0,0,4,5,NaN,NaN,0.000000,0.000000
14311,14311,2153852410,2153852410,NONE,None,None,None,Mo-Sa 06:30-18:00; Su 09:30-16:00,yes,cafe,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
14312,14312,2154222693,2154222693,NONE,None,None,None,None,yes,None,...,0,0,0,1,4,5,NaN,0.000000,0.000000,0.000000
14313,14313,2154233854,2154233854,NONE,None,None,None,None,yes,bank,...,0,0,0,2,4,5,NaN,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15961,15961,2622404476,2622404476,NONE,None,None,None,None,no,None,...,13,30,2,9,36,93,0.0,0.444444,0.361111,0.322581
23677,23677,5473274907,5473274907,NONE,None,None,None,None,no,None,...,12,30,2,10,35,93,0.0,0.400000,0.342857,0.322581
23679,23679,5473274996,5473274996,NONE,None,None,None,None,no,None,...,12,30,2,8,36,95,0.0,0.500000,0.333333,0.315789
22027,22027,4861365382,4861365382,NONE,None,None,None,None,no,None,...,12,30,2,10,35,93,0.0,0.400000,0.342857,0.322581
